### Librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import tree
from sklearn.model_selection import cross_val_score

### Datos

In [ ]:
df = pd.read_excel('Bases de datos\dataset_escalado.xlsx')
df.head()

### Pre-procesado

In [ ]:
df = df.drop(['ID_PDA','SN_PDA','FECHA_FAB','FECHA_TEST','SN_BAT'], axis=1)
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

clean_dataset(df)

### Partición y balanceo clases

In [4]:
y = df['ESTADO_BAT'].values
X = df.drop(['ESTADO_BAT'], axis=1).values

sm = SMOTE(k_neighbors=2, sampling_strategy = "auto", random_state=100)
X, y = sm.fit_resample(X, y)

In [5]:
# Modelo
modelo = tree.DecisionTreeClassifier(max_depth = 5, splitter="random", random_state=10)
# Validación cruzada
f1_tree = cross_val_score(modelo, X, y, cv=10, scoring = 'f1_weighted').mean()
# Entrenamiento
fit = modelo.fit(X, y)

In [6]:
print(f'F1 Decision Tree: {f1_tree:.5f}')

F1 Decision Tree: 0.84475


#### Ya que este modelo es interpretable procedemos a visualizarlo:

Definimos la estructura del árbol. 

In [ ]:
fig, ax = plt.subplots(figsize=(21, 10))

print(f"Profundidad del árbol: {modelo.get_depth()}")
print(f"Número de nodos terminales: {modelo.get_n_leaves()}")

plot = plot_tree(
            decision_tree = modelo,
            feature_names = list(df.drop(['ESTADO_BAT'], axis=1)),
            filled        = True,
            impurity      = False,
            fontsize      = 7,
            ax            = ax)

Nos guardamos la estructura del árbol que nos proporciona dicho modelo. 

In [ ]:
fig.savefig("árbol_clasificación.png")

Finalmente obtenemos la importacia que han tenido las variables en el modelo.  

In [ ]:
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores = pd.DataFrame(
                            {'predictor': df.drop(columns = "ESTADO_BAT").columns,
                             'importancia': modelo.feature_importances_})
importancia_predictores.sort_values('importancia', ascending=False).head(11)

### Guardar modelo

In [7]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(fit, open(filename, 'wb'))

### Probar el modelo

In [8]:
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# y = loaded_model.predict(X_test)
# print(result, y)